<a href="https://colab.research.google.com/github/Twoarms/workshop_python_machine-learning/blob/master/Parcours/2_Machine_Learning/classification_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation et import des dépendances

Nous allons utiliser, entre autre, TensorFlow Datasets, une API qui va nous permettre de télécharger et d'utiliser des ensembles de données (datasets) échantillons qu'elle met à disposition.

In [0]:
#Installation de TensorFlow Datasets sur la VM
!pip install -U tensorflow_datasets

In [0]:
#Compatibilité
from __future__ import absolute_import, division, print_function

#TensorFlow et TensorFlow Datasets
import tensorflow as tf
import tensorflow_datasets as tfd
tf.logging.set_verbosity(tf.logging.ERROR)
tf.enable_eager_execution()

#Helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt

#Améliorer affichage barre de progression
import tqdm
import tqdm.auto
tqdm.tqdm = tqdm.auto.tqdm

# Fashion MNIST

A l'origine, il y a le dataset MNIST, un jeu de donnée contenant des chiffres de 0 à 9 écrits à la main et qui est souvent le tout premier projet lorsqu'on s'initie au ML. Le Fashion MNIST a été créé pour apporter un problème légèrement plus compliqué que le MNIST classique, qui permet d'obtenir de "trop" bons résultats.

Ce dataset contient 70 000 images en niveaux de gris et de basse résolution(28*28 pixels). Il est découpé en 10 catégories de vêtements, et chaque image montre 1 article appartenant à une de ces 10 catégories.

Tout comme le MNIST classique, il a 60 000 images d'entrainement et 10 000 images de test. Ca peut paraître énorme mais ce sont des ensembles relativement petits qui sont utilisés pour vérifier qu'un algorithme fonctionne tel qu'attendu, ce qui en font des bons points de départs.

Le Fashion MNIST est accessible avec TensorFlow Datasets :

In [0]:
dataset, metadata = tfd.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

Si on doit représenter les images en termes de données, ce sont des tableaux bi-dimensionnels(28 * 28 pixels), la valeur de chaque pixel est comprise entre ```[0 et 255]``` (Car niveau de gris, si c'était du rgb, on aurait entre ```[0 et 255, 0 et 255, 0 et 255]```).

Les labels sont une liste d'integer de 0 à 9 correspondant aux classes que nous allons enregister, dans l'ordre, avec le code suivant car elles ne sont pas incluses dans le set.

In [0]:
classes = ['T-shirt/top', 'Pantalon', 'Pull', 'Robe', 'Manteau', 'Sandales', 'Chemise',
           'Baskets', 'Sac', 'Bottines']

### Afficher la répartion des données

In [0]:
number_train_examples = metadata.splits['train'].num_examples
number_test_examples = metadata.splits['test'].num_examples
print("Nombre d'exemples d'entrainement: {}".format(number_train_examples))
print("Nombre d'exemples de test: {}".format(number_test_examples))

Nous avons bien 60 000 exemples d'entrainement et 10 000 exemples de test.

Rappel : un exemple est une paire "input-output attendu", ou "feature-label"

### Pré-traitement des données

La valeur de chaque pixel est comprise entre 0 et 255, mais pour que notre modèle fonctionne correctement, cette valeur doit être comprise entre 0 et 1, nous allons donc créer une fonction et l'appliquer sur chacune des images pour convertir cette valeur.

In [0]:
def convert(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255
  return image, label

train_dataset = train_dataset.map(convert)
test_dataset = test_dataset.map(convert)

### Vérification du traitement des données

In [0]:
# Prendre une image, enlever la dimension couleur
for image, label in test_dataset.take(1):
  break
image = image.numpy().reshape((28,28))

# Tracer l'image
plt.figure()
plt.imshow(image, cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

Et voilà à quoi ressemble une image !

Nous allons maintenant afficher les 20 premières images du set d'entrainement et afficher la classe correspondante sous chaque image.

Cela nous permet de vérifier que les données sont correctement formatée et que nous sommes prêts à construire et entrainer notre modèle

In [0]:
plt.figure(figsize=(10, 10))
i = 0
for (image, label) in test_dataset.take(20):
  image = image.numpy().reshape((28,28))
  plt.subplot(5, 5, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image, cmap=plt.cm.binary)
  plt.xlabel(classes[label])
  i += 1
plt.show()

# Construire le modèle

### Préparer les couches et le modèle

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

Nous avons ici 3 couches:
1. Input layer : ```tf.keras.layers.Flatten``` - Cette couche nous permet de transformer les images qui sont des tableaux bi-dimensionnels en tableaux uni-dimensionnels. Nous avons donc un tableau de 784 (28*28) pixels au lieu d'un tableau de 28 tableaux de 28 pixels. Cette couche ne sert qu'à transformer les données.
2. "Hidden" layer : ```tf.keras.layers.Dense``` - Cette couche est "Fully Connected"* et possède 256 noeuds/neurones. Chaque neurone prend donc 1 input (chaque pixel a une seule valeur) de chacun des 784 noeuds de la couche précédente, exécute des calculs sur ces inputs fonction du poids et du biais cachés qui seront appris lors de l'entrainement et donne en output une seule valeur qui sera utilisé. 
3. Output layer : ```tf.keras.layers.Dense``` - Cette dernière couche possède 10 noeuds qui correspondent chacun à une de nos classes, chacun des 10 noeuds prend 1 input de chacun des 256 noeuds de la couche précédente, effectue les calculs avec les paramètres appris et donne en output une valeur entre 0 et 1, qui représente la probabilité que l'image appartiennent à la classe correspondante. La somme des valeurs données par chacun des 10 noeuds est donc de 1.

Qu'est-ce que [ReLu](https://www.kaggle.com/dansbecker/rectified-linear-units-relu-in-deep-learning) ? Grosso modo, il retourne 0 si une valeur est négative, sinon il retourne simplement la valeur. Cela permet de mieux capturer les effets d'interaction et les effets non-linéaires

Qu'est-ce que [Softmax](https://developers.google.com/machine-learning/crash-course/multi-class-neural-networks/softmax?hl=fr) ? Grosso modo, c'est ce qui nous permet d'avoir les probabilités qu'une image appartienne à chacune des classes pour un total de 1

### Compiler le modèle

Rappel : avant de pouvoir entrainer notre modèle, il faut le compiler et lui ajouter quelques paramètres : la fonction perte, l'optimiseur et ici nous rajoutons un paramètre de mesure qui nous permet de surveiller les étapes d'entrainement et de test. ici nous utilisons ```accuracy```, la proportion d'image correctement classifiée

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrainer le modèle

D'abord, nous déterminons l'iteration pour l'ensemble de données d'entrainement.

Nous disons à notre modèle d'utiliser des lots de 32 images, cela va influer sur le nombre d'iterations par Epoch.

Rappel : Une Epoch correspond à un cycle complet ( = Toutes les données sont passées au travers du réseau neuronal. Mais puisque notre set de données est trop trop grand pour être passé au travers du réseau en une fois, nous le découpons en lots de données)

```dataset.repeat()``` permet d'itérer "sans fin". (C'est le nombre d'Epoch qui va déterminer au final combien de temps l'entrainement va durer)

```dataset.shuffle(number_train_examples)``` randomise l'ordre des données pour que le modèle ne puisse rien apprendre de l'ordre dans lequel les données lui sont fournies. (Lui fournir les données toujours dans le même ordre l'induirait en erreur par effet mémoire)

```dataset.batch(BATCH_SIZE)``` dit simplement à la méthode ```fit``` (la méthode appelée pour entrainer le modèle) d'utiliser des batches de 32 (tel que défini au début du code)

In [0]:
BATCH_SIZE = 32
train_dataset = train_dataset.repeat().shuffle(number_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

Nous appelons ensuite la méthode ```fit``` en lui disant d'utiliser le set d'entrainement et nous définissons le nombre d'Epochs à 5 (ce qui nous donne 5 * 60 000 = 300 000 exemples pour s'entrainer).

La perte et la précision sont affichées au fur et à mesure de l'entrainement, avec une précision finale d'environ 90% sur nos données d'entrainement

In [0]:
model.fit(train_dataset, epochs=5, steps_per_epoch=math.ceil(number_train_examples/BATCH_SIZE))

# Tester la précision

Il est maintenant temps de voir comment notre modèle performe sur les données test. Nous allons donc l'évaluer sur la totalité de nos exemples de test

In [0]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(number_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)

Nous remarquons une précision un peu plus faible sur nos données test mais c'est tout a fait normal puisque notre modèle n'avait jamais été confronté à ces données, et cela rend sa précision d'autant plus impressionnante.

# Prédictions

Maintenant que notre modèle est entrainé et que nous avons évalué sa précision sur des données pour lesquelles il n'a pas été entrainé, il est venu le temps de faire des prédictions.

In [0]:
for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

In [0]:
predictions.shape

Nous avons 32 prédictions car nous avons défini plus haut que test_dataset était constitué de lots(batches) de 32 images et nous prenons 1 éléments de test_dataset : ```test_dataset.take(1)```

Chaque prédiction est constituée de 10 valeurs : la probabilité pour chaque classe de vêtement que notre image en fasse partie.

In [0]:
predictions[0]

Une prediction est un tableau de 10 nombres. Ces nombres correspondent à la certitude du modèle, la probabilité qu'une image correspond à chacune des 10 classes de vêtement (selon notre modèle). 

Nous pouvons voir de manière plus simple quelle classe a la plus grande probabilté avec le code suivant:

In [0]:
np.argmax(predictions[0])

Notre modèle pense donc que notre image appartient à la classe 6, autrement dit : Chemise.

Vérifions cela :

In [0]:
test_labels[0]

Réprésentons graphiquement l'ensemble des 10 valeurs.

D'abord nous définissons nos fonctions qui vont nous permettre cela

In [0]:
def plot_image(i, predictions_array, true_labels, images):
  predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img[...,0], cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(classes[predicted_label],
                                100*np.max(predictions_array),
                                classes[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1]) 
  predicted_label = np.argmax(predictions_array)
 
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

Maintenant regardons notre première image (index 0)

In [0]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)

Notre 13eme image (index 12)

In [0]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)

Représentons plusieurs images et leurs prédictions, le pourcentage affiché correspond à la certitude de notre modèle sur sa prédiction. Il est important de noter que le modèle peut se tromper, même avec un haut de niveau de certitude.

In [0]:
# Représenter les 15 premières images test, le label prédit et le label attendu
# La couleur bleue représente le label attendu, la couleur rouge le label prédit
# lorsqu'il ne correspond pas au label attendu
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)

Enfin, utilisons notre modèle pour une prédiction sur une seule image et analyser le résultat en détail. Reprenons pour ça notre première image (index 0), celle d'une chemise et

In [0]:
image = test_images[0]
print(image.shape)

Les modèles construits avec ```tf.keras``` sont optimisés pour des prédictions sur des lots, nous allons donc créer un "lot" d'une seule image

In [0]:
image = np.array([image])
print(image.shape)

In [0]:
unique_prediction = model.predict(image)
print(unique_prediction)

In [0]:
plot_value_array(0, unique_prediction, test_labels)

_ = plt.xticks(range(10), classes, rotation=45)

```model.predict``` retourne une liste de liste, une pour chaque image de notre lot (ici composé d'une seule image). Récupérons la prédiction :

In [0]:
np.argmax(unique_prediction[0])

Comme avant, nous avons bien le label 6 (chemise)

#### Et voilà, n'hésitez pas à
 - Jouer avec le nombre d'Epochs
 - Changer le nombre de neurones dans notre "Hidden layer"
 - Rajouter des "Hidden layers" composées de différents nombres de neurones entre la couche ```Flatten``` et la couche ```Dense``` finale (celle avec activation=tf.nn.softmax)
 
pour expérimenter un peu et obtenir des résultats différents